# GROK銘柄 イントラデイ分析

## 目的
- 今日のGROK選定銘柄（Top5/Top10/全銘柄）の前場パフォーマンス分析
- 最適売却時刻の特定（9:00寄付買い → 9:30/10:00/10:30/11:00/11:30売却）

In [4]:
# 必要なライブラリのインポート
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from datetime import datetime, time

# プロジェクトルートをパスに追加
ROOT = Path.cwd().parent
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

from common_cfg.paths import PARQUET_DIR

# Plotly設定
import plotly.io as pio
pio.templates.default = "plotly_white"

print(f"✅ ライブラリ読み込み完了")
print(f"📁 データディレクトリ: {PARQUET_DIR}")

✅ ライブラリ読み込み完了
📁 データディレクトリ: /Users/hiroyukiyamanaka/Desktop/python_stock/dash_plotly/data/parquet


## 1. データ読み込み

In [5]:
# GROK選定銘柄を読み込み
grok_file = PARQUET_DIR / "grok_trending.parquet"
df_grok = pd.read_parquet(grok_file)

print(f"GROK選定銘柄数: {len(df_grok)}")
print(f"選定日時: {df_grok['selected_time'].iloc[0] if 'selected_time' in df_grok.columns else 'N/A'}")
print(f"\nTop5銘柄:")
print(df_grok.nsmallest(5, 'grok_rank')[['ticker', 'stock_name', 'selection_score', 'grok_rank']])

# 分析対象日（今日）
TARGET_DATE = datetime.now().strftime("%Y-%m-%d")
print(f"\n📅 分析対象日: {TARGET_DATE}")

GROK選定銘柄数: 12
選定日時: 23:00

Top5銘柄:
   ticker         stock_name  selection_score  grok_rank
4  3911.T             Aiming            148.0          1
1  4592.T              サンバイオ            160.0          2
8  7078.T          INCLUSIVE             88.0          3
3  4417.T  グローバルセキュリティエキスパート            155.0          4
6  3936.T           グローバルウェイ             90.0          5

📅 分析対象日: 2025-10-27


In [ ]:
# 5分足データを読み込み
prices_5m_file = PARQUET_DIR / "prices_60d_5m.parquet"
df_prices = pd.read_parquet(prices_5m_file)

print(f"✅ 5分足データ読み込み完了")
print(f"データ期間: {df_prices['date'].min()} 〜 {df_prices['date'].max()}")
print(f"銘柄数: {df_prices['ticker'].nunique()}")
print(f"データ件数: {len(df_prices):,}")

## 2. 当日データの抽出と前処理

In [ ]:
# 当日のGROK銘柄の価格データのみ抽出
grok_tickers = df_grok['ticker'].tolist()

# 当日データをフィルタ
df_today = df_prices[
    (df_prices['ticker'].isin(grok_tickers)) &
    (df_prices['date'].dt.date == pd.to_datetime(TARGET_DATE).date())
].copy()

print(f"当日のGROK銘柄データ: {len(df_today):,}件")
print(f"銘柄数: {df_today['ticker'].nunique()} / {len(grok_tickers)}")

if len(df_today) == 0:
    print(f"\n⚠️ 当日のデータがありません。分析対象日を確認してください。")
else:
    # 時刻列を追加
    df_today['time'] = df_today['date'].dt.time
    df_today['hour_minute'] = df_today['date'].dt.strftime('%H:%M')
    
    print(f"\n時刻範囲: {df_today['hour_minute'].min()} 〜 {df_today['hour_minute'].max()}")
    print(f"\nサンプルデータ:")
    print(df_today[['ticker', 'date', 'hour_minute', 'Open', 'Close', 'Volume']].head(10))

## 3. 寄付価格と各時刻での利益計算

In [ ]:
# 分析対象時刻の定義
TIME_SLOTS = {
    '09:00': '寄付（買い）',
    '09:30': '9:30売却',
    '10:00': '10:00売却',
    '10:30': '10:30売却',
    '11:00': '11:00売却',
    '11:30': '前引け売却',
}

def get_price_at_time(df, ticker, target_time):
    """指定時刻の価格を取得（前後5分の範囲で最も近い時刻）"""
    df_ticker = df[df['ticker'] == ticker].copy()
    if len(df_ticker) == 0:
        return None
    
    # 目標時刻の前後5分の範囲でデータを探す
    target_dt = datetime.strptime(target_time, '%H:%M').time()
    df_ticker['time_diff'] = df_ticker['date'].apply(
        lambda x: abs((x.time().hour * 60 + x.time().minute) - 
                     (target_dt.hour * 60 + target_dt.minute))
    )
    
    # 10分以内の最も近い時刻のデータを取得
    closest = df_ticker[df_ticker['time_diff'] <= 10].nsmallest(1, 'time_diff')
    
    if len(closest) > 0:
        return closest['Close'].iloc[0]
    return None

# 各銘柄の各時刻での価格を取得
results = []

for ticker in grok_tickers:
    row_data = {'ticker': ticker}
    
    # GROK情報を追加
    grok_info = df_grok[df_grok['ticker'] == ticker].iloc[0]
    row_data['stock_name'] = grok_info['stock_name']
    row_data['selection_score'] = grok_info['selection_score']
    row_data['grok_rank'] = grok_info['grok_rank']
    row_data['is_top5'] = grok_info['grok_rank'] <= 5
    row_data['is_top10'] = grok_info['grok_rank'] <= 10
    
    # 各時刻の価格を取得
    for time_str in TIME_SLOTS.keys():
        price = get_price_at_time(df_today, ticker, time_str)
        row_data[f'price_{time_str}'] = price
    
    results.append(row_data)

df_results = pd.DataFrame(results)

# 寄付価格（買値）を基準に各時刻での利益率を計算
buy_price_col = 'price_09:00'

for time_str in ['09:30', '10:00', '10:30', '11:00', '11:30']:
    sell_price_col = f'price_{time_str}'
    return_col = f'return_{time_str}'
    
    df_results[return_col] = (
        (df_results[sell_price_col] - df_results[buy_price_col]) / 
        df_results[buy_price_col] * 100
    )

print(f"✅ 利益率計算完了")
print(f"\n価格データがある銘柄数: {df_results[buy_price_col].notna().sum()} / {len(df_results)}")

# データがある銘柄のみにフィルタ
df_results_valid = df_results[df_results[buy_price_col].notna()].copy()

print(f"\nサンプル（Top5銘柄）:")
display_cols = ['ticker', 'stock_name', 'grok_rank', buy_price_col] + \
               [f'return_{t}' for t in ['09:30', '10:00', '10:30', '11:00', '11:30']]
print(df_results_valid[df_results_valid['is_top5']][display_cols].head())

## 4. 集計統計の計算

In [ ]:
# グループ別の統計を計算
def calc_group_stats(df, group_name):
    """グループの統計を計算"""
    stats = {'group': group_name, 'count': len(df)}
    
    for time_str in ['09:30', '10:00', '10:30', '11:00', '11:30']:
        return_col = f'return_{time_str}'
        
        if return_col in df.columns:
            valid_returns = df[return_col].dropna()
            
            if len(valid_returns) > 0:
                stats[f'{time_str}_avg'] = valid_returns.mean()
                stats[f'{time_str}_win_rate'] = (valid_returns > 0).sum() / len(valid_returns) * 100
                stats[f'{time_str}_count'] = len(valid_returns)
            else:
                stats[f'{time_str}_avg'] = None
                stats[f'{time_str}_win_rate'] = None
                stats[f'{time_str}_count'] = 0
    
    return stats

# 各グループの統計
stats_top5 = calc_group_stats(df_results_valid[df_results_valid['is_top5']], 'Top5')
stats_top10 = calc_group_stats(df_results_valid[df_results_valid['is_top10']], 'Top10')
stats_all = calc_group_stats(df_results_valid, '全銘柄')

df_stats = pd.DataFrame([stats_top5, stats_top10, stats_all])

print("\n📊 グループ別統計:")
print(df_stats[['group', 'count', '11:30_avg', '11:30_win_rate']].to_string(index=False))

## 5. ビジュアライゼーション

In [ ]:
# グラフ1: 時刻別平均リターン（グループ比較）
time_points = ['09:30', '10:00', '10:30', '11:00', '11:30']
time_labels = ['9:30', '10:00', '10:30', '11:00', '11:30']

fig_returns = go.Figure()

for idx, row in df_stats.iterrows():
    returns = [row[f'{t}_avg'] for t in time_points]
    
    fig_returns.add_trace(go.Scatter(
        x=time_labels,
        y=returns,
        mode='lines+markers',
        name=row['group'],
        line=dict(width=3),
        marker=dict(size=10)
    ))

fig_returns.add_hline(y=0, line_dash="dash", line_color="gray", opacity=0.5)

fig_returns.update_layout(
    title=f'<b>時刻別平均リターン（{TARGET_DATE}）</b><br><sub>9:00寄付買い → 各時刻売却</sub>',
    xaxis_title='売却時刻',
    yaxis_title='平均リターン (%)',
    height=500,
    hovermode='x unified',
    legend=dict(x=0.02, y=0.98, bgcolor='rgba(255,255,255,0.8)')
)

fig_returns.show()

In [ ]:
# グラフ2: 時刻別勝率（グループ比較）
fig_winrate = go.Figure()

for idx, row in df_stats.iterrows():
    win_rates = [row[f'{t}_win_rate'] for t in time_points]
    
    fig_winrate.add_trace(go.Bar(
        x=time_labels,
        y=win_rates,
        name=row['group'],
        text=[f'{v:.1f}%' if v is not None else 'N/A' for v in win_rates],
        textposition='outside'
    ))

fig_winrate.add_hline(y=50, line_dash="dash", line_color="gray", opacity=0.5,
                       annotation_text="50%（損益分岐）")

fig_winrate.update_layout(
    title=f'<b>時刻別勝率（{TARGET_DATE}）</b><br><sub>9:00寄付買い → 各時刻売却</sub>',
    xaxis_title='売却時刻',
    yaxis_title='勝率 (%)',
    height=500,
    barmode='group',
    hovermode='x unified',
    yaxis=dict(range=[0, 100]),
    legend=dict(x=0.02, y=0.98, bgcolor='rgba(255,255,255,0.8)')
)

fig_winrate.show()

In [ ]:
# グラフ3: 個別銘柄のリターン（Top5のみ）
df_top5 = df_results_valid[df_results_valid['is_top5']].copy()

if len(df_top5) > 0:
    fig_individual = go.Figure()
    
    for _, row in df_top5.iterrows():
        returns = [row[f'return_{t}'] for t in time_points]
        
        fig_individual.add_trace(go.Scatter(
            x=time_labels,
            y=returns,
            mode='lines+markers',
            name=f"{row['ticker']} {row['stock_name']}",
            hovertemplate='%{y:.2f}%<extra></extra>'
        ))
    
    fig_individual.add_hline(y=0, line_dash="dash", line_color="gray", opacity=0.5)
    
    fig_individual.update_layout(
        title=f'<b>Top5銘柄 個別リターン（{TARGET_DATE}）</b><br><sub>9:00寄付買い → 各時刻売却</sub>',
        xaxis_title='売却時刻',
        yaxis_title='リターン (%)',
        height=600,
        hovermode='x unified',
        legend=dict(x=1.02, y=1, bgcolor='rgba(255,255,255,0.8)')
    )
    
    fig_individual.show()
else:
    print("⚠️ Top5銘柄のデータがありません")

## 6. 詳細データテーブル

In [ ]:
# Top5銘柄の詳細テーブル
if len(df_top5) > 0:
    print("\n📋 Top5銘柄の詳細:")
    print("="*120)
    
    display_df = df_top5.sort_values('grok_rank')[[
        'grok_rank', 'ticker', 'stock_name', 'selection_score',
        'price_09:00', 'return_09:30', 'return_10:00', 'return_10:30',
        'return_11:00', 'return_11:30'
    ]].copy()
    
    # 数値フォーマット
    for col in display_df.columns:
        if 'return_' in col:
            display_df[col] = display_df[col].apply(lambda x: f"{x:+.2f}%" if pd.notna(x) else "N/A")
        elif col == 'price_09:00':
            display_df[col] = display_df[col].apply(lambda x: f"{x:.0f}" if pd.notna(x) else "N/A")
    
    display_df.columns = [
        'ランク', 'コード', '銘柄名', 'スコア', '寄付',
        '9:30', '10:00', '10:30', '11:00', '11:30(前引)'
    ]
    
    print(display_df.to_string(index=False))
    print("="*120)

## 7. サマリーレポート

In [ ]:
print(f"\n" + "="*80)
print(f"📊 GROK銘柄 前場パフォーマンスサマリー ({TARGET_DATE})")
print("="*80)

print(f"\n【戦略】9:00 寄付買い → 前引け（11:30）売却")
print("-"*80)

for _, row in df_stats.iterrows():
    group = row['group']
    count = row['11:30_count']
    avg_return = row['11:30_avg']
    win_rate = row['11:30_win_rate']
    
    print(f"\n{group} ({count}銘柄):")
    if avg_return is not None:
        print(f"  平均リターン: {avg_return:+.2f}%")
        print(f"  勝率: {win_rate:.1f}%")
    else:
        print(f"  データなし")

# 最適売却時刻の特定
print(f"\n\n【最適売却時刻分析】")
print("-"*80)

for group in ['Top5', 'Top10', '全銘柄']:
    group_data = df_stats[df_stats['group'] == group].iloc[0]
    
    best_time = None
    best_return = -999
    
    for t in time_points:
        ret = group_data[f'{t}_avg']
        if ret is not None and ret > best_return:
            best_return = ret
            best_time = t
    
    if best_time:
        print(f"\n{group}: {best_time} ({best_return:+.2f}%)")
    else:
        print(f"\n{group}: データ不足")

print("\n" + "="*80)

## 8. HTMLエクスポート（オプション）

グラフをHTMLファイルとして保存してブラウザで開けます。

In [ ]:
# HTMLファイルとして保存
output_dir = ROOT / "output" / "grok_analysis"
output_dir.mkdir(parents=True, exist_ok=True)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

fig_returns.write_html(output_dir / f"returns_{timestamp}.html")
fig_winrate.write_html(output_dir / f"winrate_{timestamp}.html")

if len(df_top5) > 0:
    fig_individual.write_html(output_dir / f"top5_individual_{timestamp}.html")

print(f"\n✅ HTMLファイルを保存しました: {output_dir}")
print(f"\nブラウザで開くには:")
print(f"  open {output_dir}/returns_{timestamp}.html")